In [ ]:
function effective_weekday(Arg dataframe) {
    dataframe['effective_weekday'] = Use dayofweek function from datetime package to turn "effective_date" into a 0-6 value, 
    0 for monday, 6 for sunday
}
function calculate_deltas(Arg dataframe, Arg qc) {

    copy = copy of dataframe where deep = True 
    copy[“prev_entries”] = Group copy by columns “unit” and “scp”, extract“entries” column and shift by one row
    copy[“prev_exits”] =  Group copy by columns “unit” and “scp”, extract “exits” column and shift by one row 
    copy[“entry_delta”] = Calculate difference between columns “entries” and “prev_entries”
    copy[“exit_delta”] = Calculate difference between columns “exits” and “prev_exits” 

    if qc == True
        qc = Call function qc_deltas on copy
        deltas = Drop null values in qc for columns “entry_delta” and “exit_delta”, and reset the index 
    else 
        print “qc = False set to false so qc skipped”
        deltas = copy 

    return deltas}


function qc_deltas(arg dataframe) {

    qc_df = calls fix_negatives function from df
    
    outliers = subset of qc dataframe where "entry_delta" or "exit_delta" is greater than 3000 is an outlier

    good_data = subset of qc dataframe where "entry_delta" and "exit_delta” is between 0 and 3000, inclusive

    for index, row in outliers
        for each remaining outlier, determine if it is an outlier 
    return qc_df}


function fix_backwards(Arg dataframe) {

    entry = Subset of dataframe where “entry_delta” is between 0 and -2000, non-inclusive
    exit = Subset of dataframe where “exit_delta” is between 0 and -2000, non-inclusive

    entry_counts = Group “entry” by columns “unit” and “scp”, count number of stations, reset index to “count”, subset columns “unit”, “scp”, “count”
    exit_counts = Group “exit” by columns “unit” and “scp”, count number of stations, reset index to “count”, subset columns “unit”, “scp”, “count”

    copy = copy of dataframe where deep = True 

    for index and row in entry_counts 
        if “count” column in row > 10 
            mask = Boolean list of whether (“unit” column in “entry” == “unit” column in “row”) and (“scp” column in “entry” == “scp” column in “row”)
            valid = List of indices in “entry” where index in mask 
            abs_values = Absolute values of “entry_delta” column in “entry” where index in mask
            Subset “copy” where values in “entry_delta” column with index in valid == abs_values 

    for index and row in exit_counts 
        if “count” column in row > 10 
            mask = Boolean list of whether (“unit” column in “exit” == “unit” column in “row”) and (“scp” column in “exit” == “scp” column in “row”)
            valid = List of indices in “exit” where index in mask 
            abs_values = Absolute values of “exit_delta” column in “exit” where index in mask
            Subset “copy” where values in “exit_delta” column with index in valid == abs_values 

    return copy}


function fix_negatives(arg dataframe) {
    copy = call fix_backwards function on arg

    negatives = Subset 'copy' Dataframe with rows only where 'entry_delta' < 0 or 'exit_data' < 0
    good_data = Subset arg Dataframe where 'entry_delta' is between 0 and 3000, inclusive and 'exit_delta' is between 0 and 3000, inclusive

    For index, row in ‘negatives’ Dataframe:
        Subset 'copy' where values in 'entry_delta' column with index, Subset 'copy' where values in 'exit_data' column with index == fix_outlier(row, good_data)

    return copy}


function fix_outlier(Arg row, Arg good_data) { 

    rel = Subset “good_data” where (“unit’ column in “good_data” == “unit” column in “row”) and (“scp” column in “good_data” == “scp” column in “row)

    if number of rows in “rel” > 0
        entry_mean = Mean of the “entry_delta” column in “rel” 
        exit_mean = Mean of the “exit_delta” column in “rel”

        if number of rows in rel > 1
            entry_limit = “entry_mean” + Standard deviation of “entry_delta” column in “rel” multiplied by 3 
            exit_limit = “exit_mean” + Standard deviation of “exit_delta” column in “rel” multiplied by 3 
        else 
            entry_limit = “entry_mean”
            exit_limit = “exit_mean”

        if absolute value of “entry_delta” column in “row” > “entry_limit”
            entry_delta = “entry_mean”
        else 
            entry_delta = Absolute value of “entry_delta” column in “row” 

        if absolute value of “exit_delta” column in “row” > “exit_limit”
            exit_delta = “exit_mean”
        else 
            exit_delta = Absolute value of “exit_delta” column in “row” 

        return “entry_delta”, “exit_delta”
    else 
        return None, None 



function aggregate_to_station(deltas) {

#deltas is the Dataframe returned by calculate_deltas()

    grouped_by = Group deltas Dataframe by 'unit', 'station', 'effective_date', Sum deltas by groups & reset index
    station_agg = Subset of grouped_by Dataframe ("unit", "station", "effective_date", "entry_delta", "exit_delta" columns)

    return station_agg}


function pick_select_stations(Arg station_agg, Arg boro_mapping, Arg date, Arg select_stations, Arg replace_map) {

    boro_station = Merge “station_agg” to “boro_mapping” with left join on columns “unit” and “station”
    Replace “station” column in “boro_station” with “replace_map” so regex= True and inplace= True
    indexes = Create empty list 

    for “boro” and “stations” in “select_stations” items 
        for “station” in “stations”
            Add to “indexes” the index of rows in subset of “boro_station” where (“station” of “boro_station” == “station”) and (“borough” of “boro_station” == “boro”)

    indexes = List of set of “indexes”
    selected_agg = Subset “boro_station” where indexes are in “indexes” 
    selected_agg = Group by columns “station”, “borough”, “effective_date”, sum 
    columns, reset index, and only include columns “station”, “borough”, “effective_date”, “entry_delta”, “exit_delta”

    Round “entry_delta” column in “selected_agg” to whole number
    Round “exit_delta” column in “selected_agg” to whole number 

    enddate = Transform “date” into datetime format “%y%m%d” 
    startdate = Subtract timedelta of value 7 from “enddate”
    Drop rows inplace from “selected_agg” where “effective_date” < startdate

    days_check = Group “selected_agg” by columns “station“ and “borough”, count other columns, and reset index
    missing_days = “station” values where “effective_date” of “days_check” != 7
    if shape of  “missing_days” != 0 
        logging.error(missing_days + “ does not have 7 days in” + date + “ file.”)

    week = Group “selected_agg” by columns “station” and “borough”, sum columns, reset index 
    Assign to “effective_date” column timedelta of 1 subtracted from “enddate”
    Assign to “borough” column value in “borough” concatenated with string “W” 
    Reorder “week” columns to match “boro_agg”

    selected_agg = Concatenate “selected_agg” and “week”, reset index so drop = True 

    return selected_agg}


function aggregate_to_borough(Arg station_agg, Arg boro_mapping, Arg date) {

    boro = Merge “station_agg” to “boro_mapping” with left join on columns “unit” and “station”

    if number of null values in “boro” columns != 0
        logging.error (“Failed to map” + unique “station” in subset of “boro” where columns have null values)
        Drop null values inplace in “boro”

    boro_agg = Group “boro” by columns “borough” and “effective_date”, sum “entry_delta” and “exit_delta” columns, reset index 

    Round “entry_delta” column in “boro_agg” to whole number 
    Round “exit_delta” column in “boro_agg” to whole number 

    enddate = Transform “date” into datetime format “%y%m%d” 
    startdate = Subtract timedelta of value 7 from “enddate”
    Drop rows inplace from “boro_agg” where “effective_date” < startdate

    all_nyc = Group “boro_agg” by “effective_date” column, sum columns “entry_delta” and “exit_delta”, reset index
    Assign “ALL” value to “borough” column
    Reorder “all_nyc” columns to match “boro_agg”

    boro_agg = Concatenate “boro_agg” and “all_nyc” dataframes, reset index so drop = True

    days_check = Group “boro_agg” by “borough” column, count other columns, and reset index
    missing_days = “borough” values where “effective_date” of “days_check” != 7
    if shape of  “missing_days” != 0 
        logging.error(missing_days + “ does not have 7 days in” + date + “ file.”)

    week = Group “boro_agg” by “borough”, sum columns, reset index 
    Assign to “effective_date” column timedelta of 1 subtracted from “enddate”
    Assign to “borough” column value in “borough” concatenated with string “W” 
    Reorder “week” columns to match “boro_agg”

    boro_agg = Concatenate “boro_agg” and “week” tables, reset index so drop = True 

    return boro_agg}
